In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
    --------------------------------------- 0.8/38.8 MB 3.0 MB/s eta 0:00:13
   - -------------------------------------- 1.6/38.8 MB 3.2 MB/s eta 0:00:12
   -- ------------------------------------- 2.4/38.8 MB 3.2 MB/s eta 0:00:12
   -- ------------------------------------- 2.9/38.8 MB 3.2 MB/s eta 0:00:12
   --- ------------------------------------ 3.4/38.8 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 4.5/38.8 MB 3.2 MB/s eta 0:00:11
   ----- ---------------------------------- 5.0/38.8 MB 3.3 MB/s eta 0:00:11
   ----- ---------------------------------- 5.8/38.8 MB 3.3 MB/s eta 0:00:11
   ------ --------------------------------- 6.6/38.8 MB 3.3 MB/s eta 0:00:10
   ------

In [2]:
!pip install roboflow

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import cv2
from roboflow import Roboflow
import matplotlib.pyplot as plt
import numpy as np
import socket
import threading

In [19]:
import cv2
from roboflow import Roboflow  # Make sure you have roboflow installed

SERVER_ADDRESS = 'DESKTOP-1RLK4BP'  # Replace with the actual server address
SERVER_PORT = 8000

# Function to handle receiving messages from the server
def receive_messages(sock):
    while True:
        try:
            message = sock.recv(1024).decode('utf-8')
            #if message:
                #print(f"Received from server: {message}")
            #else:
                #break
        except Exception as e:
            print(f"Error receiving message: {e}")
            break

# Function to start the client and connect to the server
def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        client_socket.connect((SERVER_ADDRESS, SERVER_PORT))
        print("Connected to the server.")

        # Start a thread to listen for messages from the server
        threading.Thread(target=receive_messages, args=(client_socket,), daemon=True).start()

        return client_socket

    except Exception as e:
        print(f"Failed to connect to the server: {e}")
        return None

# Initialize Roboflow with the API key
rf = Roboflow(api_key="xz8xVdWOboKdOaCGw2mz")  # Replace with actual API key
project = rf.workspace().project("hci-objects")  # Replace with project name
model = project.version(1).model  # Adjust version if needed

client_socket = start_client()  
# Open the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    # Get predictions from the model
    results = model.predict(frame, confidence=40, overlap=30).json()

    # Draw bounding boxes on the frame
    for prediction in results['predictions']:
        x_min = int(prediction['x'] - prediction['width'] / 2)
        y_min = int(prediction['y'] - prediction['height'] / 2)
        x_max = int(prediction['x'] + prediction['width'] / 2)
        y_max = int(prediction['y'] + prediction['height'] / 2)
        centerX = (x_max + x_min)/2
        centerY = (y_max + y_min)/2
        label = prediction['class']
        print(label)
        if client_socket:
            try:
                message = f"SENDTO:1|{label}, {centerX}, {centerY}"
                client_socket.send(message.encode('utf-8'))
                print(f"{message}")
            except Exception as e:
                print(f"Error sending message: {e}")
        confidence = prediction['confidence']

        # Draw rectangle and label
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with predictions
    cv2.imshow("Roboflow Object Detection", frame)

    # Exit loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


loading Roboflow workspace...
loading Roboflow project...
Connected to the server.
egypt
SENDTO:1|egypt, 279.0, 214.0
egypt
SENDTO:1|egypt, 312.0, 218.5
egypt
SENDTO:1|egypt, 285.5, 213.5
egypt
SENDTO:1|egypt, 276.0, 215.0
egypt
SENDTO:1|egypt, 290.5, 214.0
egypt
SENDTO:1|egypt, 285.5, 214.0
egypt
SENDTO:1|egypt, 289.5, 216.5
egypt
SENDTO:1|egypt, 294.0, 211.0
egypt
SENDTO:1|egypt, 292.0, 209.5


Error receiving message: [WinError 10054] An existing connection was forcibly closed by the remote host
